In [1]:
from src.utils import *

In [2]:
# delete 75% of files in the directory
import os
import random

def delete_files(directory, percentage=0.75):
    files = os.listdir(directory)
    files = random.sample(files, int(len(files) * percentage))
    for file in files:
        os.remove(os.path.join(directory, file))

# delete_files('data/train/audio_yes_no/no', 0.5)
# delete_files('data/train/audio_yes_no/yes', 0.5)

In [3]:
import torch
from transformers import AutoFeatureExtractor, ASTForAudioClassification

model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

In [4]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
import os

class CustomAudioDataset(Dataset):
    def __init__(self, data_dir, transform=None, fixed_length=None):
        self.data_dir = data_dir
        self.file_list, self.labels = self._get_file_list_and_labels()
        self.transform = transform
        self.fixed_length = fixed_length

    def _get_file_list_and_labels(self):
        file_list = []
        labels = []
        for root, dirs, files in os.walk(self.data_dir):
            for file in files:
                if file.endswith(".wav"):  # Adjust file extension if needed
                    file_list.append(root + "/" + file)
                    labels.append(os.path.basename(root))  # Extract label from directory name
        return file_list, labels

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        waveform, sample_rate = torchaudio.load(file_path)
        
        
        
        if self.fixed_length:
            waveform = self._pad_waveform(waveform, self.fixed_length)

        label = self.labels[idx]

        

        if self.transform:
            waveform = self.transform(waveform, sampling_rate=sample_rate)['input_values'][0]

        return waveform, sample_rate, label

    def _pad_waveform(self, waveform, target_length):
        length_diff = target_length - waveform.size(1)
        if length_diff > 0:
            padding = torch.zeros((1, length_diff))
            waveform = torch.cat([waveform, padding], dim=1)
        return waveform.squeeze(0)

# Example usage
data_dir = "data/train/audio_small/"
# data_dir = "data/train/audio_yes_no/"
transform = feature_extractor  # You can define transformations if needed
fixed_length = 16000  # Assuming you want to fix the length to 16000 samples
sampling_rate = fixed_length  # Assuming you want to fix the sampling rate to 16000 Hz



batch_size = 4

In [5]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def freeze_layers_except_last_n(model, n):
    # Get all parameters
    parameters = list(model.parameters())
    total_layers = len(parameters)

    # Freeze all layers except the last n
    for i, param in enumerate(parameters):
        if i < total_layers - n:
            param.requires_grad = False




In [6]:
num_epochs = 5
perc = 0.05

In [7]:
# set number of output classes
num_classes = 11


In [8]:
import random
import numpy as np

for i in tqdm(np.arange (2, 5, 1), desc='Training loop (5 times)'):
   
    random.seed(int(i))
    torch.manual_seed(i)
    torch.cuda.manual_seed(i)
    torch.cuda.manual_seed_all(i)

    
   
    model = ASTForAudioClassification.from_pretrained(model_name)

    only_name = model_name.split("/")[-1]   


    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()

    device = torch.device('cuda')
    # Assuming your model is named ast_model
    freeze_layers_except_last_n(model.audio_spectrogram_transformer.encoder.layer, 4)

    model.classifier.dense = torch.nn.Linear(768, num_classes, bias=True)

    model.to(device)

    train_dataset = CustomAudioDataset(data_dir, fixed_length=16000, transform=feature_extractor)
                                    


    n_train = len(train_dataset)
    n_val = int(perc * n_train)
    n_test = n_val//2
    n_train = n_train - n_val

    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [n_train, n_val], generator=torch.Generator().manual_seed(42))
    val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [n_val-n_test, n_test], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    

    labels = set(train_dataset.dataset.labels)
    label_to_index = dict((label, i) for i, label in enumerate(sorted(labels)))
    name = data_dir.split("/")[-2]
    # print(name)
    log_dir = train(model, train_loader, val_loader, num_epochs, optimizer, criterion, device, label_to_index, only_name, log=True, description=f"no_freeze_test_{name}_{i}")
    test(model, test_loader, criterion, device, label_to_index, only_name, log_dir)



Training loop (5 times):   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/6187 [00:00<?, ?it/s]

Epoch 1/5, Train Loss: 1.1480810248789879


Validation:   0%|          | 0/163 [00:00<?, ?it/s]

Epoch 1/5, Validation Loss: 0.6778689024090035, Validation Accuracy: 0.7864823348694316


...

Epoch 5/5, Train Loss: 0.30547947467271497


Validation:   0%|          | 0/163 [00:00<?, ?it/s]

Epoch 5/5, Validation Loss: 0.41778519140853565, Validation Accuracy: 0.869431643625192


Testing:   0%|          | 0/163 [00:00<?, ?it/s]

{'val_acc': [0.7603686635944701, 0.7757296466973886, 0.8571428571428571, 0.8448540706605223, 0.869431643625192], 'val_loss': [0.6752864158043832, 0.6282782801341242, 0.4615609240203959, 0.47299084306888833, 0.41778519140853565], 'train_loss': [1.0521215107734492, 0.5544123916581324, 0.4320718958473075, 0.35905831246163394, 0.30547947467271497], 'test_correct_in_batch': [4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 1.0, 4.0, 3.0, 4.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 2.0, 4.0, 2.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 3.0, 4.0, 1.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 1.0, 3.0, 3.0, 4.0, 4.0, 2.0, 4.0, 4.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 2.0, 4.0, 2.0, 4.0, 4.0, 2.0, 4.